In [1]:
# import libraries
'''try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass'''
import tensorflow as tf
import pandas as pd
import keras
#!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences

print(tf.__version__)

2.15.0


In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
valid_file_path = "valid-data.tsv"

train_data = pd.read_table(train_file_path, names=['type', 'text'])
test_data = pd.read_table(valid_file_path, names=['type', 'text'])

train_data.head()

--2024-01-10 23:03:49--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K   618KB/s    in 0.6s    

2024-01-10 23:03:51 (618 KB/s) - ‘train-data.tsv’ saved [358233/358233]

--2024-01-10 23:03:51--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[=============

,type,text
0,ham,ahhhh...just woken up!had a bad dream about u ...
1,ham,you can never do nothing
2,ham,"now u sound like manky scouse boy steve,like! ..."
3,ham,mum say we wan to go then go... then she can s...
4,ham,never y lei... i v lazy... got wat? dat day ü ...


In [3]:
#let's create a function where we can transform arrays of characters to an array with number
vocab = {}
word_encoding = 1
def one_hot_encoding(array):
  global word_encoding
  for j in range(len(array)):
    text = array.iloc[j]
    words = text.lower().split(" ")
    encoding = []
    for word in words:
      if word in vocab:
        code = vocab[word]
        encoding.append(code)
      else:
        vocab[word] = word_encoding
        encoding.append(word_encoding)
        word_encoding += 1
    array.iloc[j] = encoding
  return array

In [4]:
#let's create the train data and valid data
train_label = train_data.pop('type')
train_label = train_label.replace(['ham','spam'],[0,1])
train_label = np.array(train_label, dtype=int)
train_data = one_hot_encoding(train_data['text'])
train_data = np.array(train_data )

test_label = test_data.pop('type')
test_label = test_label.replace(['ham','spam'],[0,1])
test_label = np.array(test_label, dtype=int )
test_data = one_hot_encoding(test_data['text'])
test_data = np.array(test_data)

In [5]:
#let's have a look of the train data
train_data, train_label

(array([list([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 8, 13, 14, 15, 10, 16, 17, 18, 7, 19, 20, 21, 10, 22, 23, 24, 25, 26]),
        list([27, 28, 29, 30, 31]),
        list([14, 8, 32, 12, 33, 34, 35, 36, 10, 37, 38, 39, 40, 41, 42, 43, 8, 44, 45, 46, 47, 48]),
        ...,
        list([487, 885, 886, 198, 3205, 830, 2689, 148, 353, 84, 3720, 971, 53, 9663, 258, 1088, 729, 9664]),
        list([11323, 1239, 11324, 11325, 37, 1768, 27, 53, 277, 447, 768, 177, 11326, 129, 11327, 154, 1771, 11328, 1238, 72, 1239, 1773, 53, 1240]),
        list([83, 90, 213, 5, 10912, 421, 8, 322, 3576, 78, 273, 1172, 641, 53, 1775, 78, 213, 214, 808, 10, 4550, 7632, 4719, 190, 90, 513, 1584, 11329, 1142, 25, 182, 11330, 11331, 3328])],
       dtype=object),
 array([0, 0, 0, ..., 1, 1, 0]))

In [6]:
#padding
MAXLEN = 40
train_data = keras.utils.pad_sequences(train_data , MAXLEN)
test_data = keras.utils.pad_sequences(test_data , MAXLEN)

In [7]:
#the vocabulary_size should be big enough so the training, valid and predictions data can use the model
vocabulary_size = np.unique(train_data)[-1]+4000
vocabulary_size

15331

In [8]:
#model
#emmbedding and LSTM to give sentimental closseness to the words
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocabulary_size, 64),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(64, activation="sigmoid"),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 64)          981184    
                                                                 
 lstm (LSTM)                 (None, 64)                33024     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1018433 (3.89 MB)
Trainable params: 1018433 (3.89 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [9]:
#training
history = model.fit(train_data, train_label, epochs=6, steps_per_epoch=100)

Epoch 1/6
100/100 [==============================] - 18s 124ms/step - loss: 0.2475 - accuracy: 0.9091
Epoch 2/6
100/100 [==============================] - 7s 67ms/step - loss: 0.0424 - accuracy: 0.9885
Epoch 3/6
100/100 [==============================] - 4s 36ms/step - loss: 0.0174 - accuracy: 0.9971
Epoch 4/6
100/100 [==============================] - 3s 27ms/step - loss: 0.0082 - accuracy: 0.9988
Epoch 5/6
100/100 [==============================] - 2s 20ms/step - loss: 0.0049 - accuracy: 0.9993
Epoch 6/6
100/100 [==============================] - 2s 24ms/step - loss: 0.0034 - accuracy: 0.9995


In [10]:
#evaluate the data with the test data
results = model.evaluate(test_data, test_label)

44/44 [==============================] - 1s 5ms/step - loss: 0.0525 - accuracy: 0.9849


In [11]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
#the function transform the text into numeric data, then applies the padding and make the prediction
def predict_message(text):
  text = pd.Series(text)
  text = one_hot_encoding(text)
  encoded_text = keras.utils.pad_sequences(text , MAXLEN)
  result = model.predict(encoded_text)
  if result[0][0] >= 0.5:
    return[result[0][0],'spam']
  else:
    return[result[0][0],'ham']

pred_text = "sale today! to stop texts call 98912460324"

prediction = predict_message(pred_text)
print(prediction)

1/1 [==============================] - 1s 555ms/step
[0.99055636, 'spam']


In [12]:
# Run this cell to test your function and model. Do not modify contents.
#Cell to know if the project passes the test
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 [==============================] - 0s 24ms/step
You passed the challenge. Great job!
